# Helmholtz solver using Burton Miller formulation

https://github.com/bempp/bempp-cl/blob/main/notebooks/helmholtz/helmholtz_combined_exterior.ipynb

In [ ]:
from netgen.occ import *
from ngsolve import *
from ngsolve.webgui import Draw
from ngsolve.bem import *
from ngsolve import Projector, Preconditioner
from ngsolve.krylovspace import CG, GMRes

In [ ]:
order=4

In [ ]:
screen = WorkPlane(Axes( (0,0,0), Z, X)).RectangleC(15,15).Face()

sp = Fuse(Sphere( (0,0,0), pi).faces)
screen.faces.name="screen"
sp.faces.name="sphere"
shape = Compound([screen,sp])
# Draw (shape)
mesh = Mesh( OCCGeometry(shape).GenerateMesh(maxh=0.5)).Curve(order)
Draw (mesh);

In [ ]:
fes_sphere = Compress(SurfaceL2(mesh, order=order, dual_mapping=False, complex=True, definedon=mesh.Boundaries("sphere")))
fes_sphere_test = Compress(SurfaceL2(mesh, order=order, dual_mapping=False, complex=True, definedon=mesh.Boundaries("sphere")))
fes_screen = Compress(SurfaceL2(mesh, order=order, dual_mapping=True, complex=True, definedon=mesh.Boundaries("screen")))
print ("ndof_sphere = ", fes_sphere.ndof, "ndof_screen =", fes_screen.ndof)

In [ ]:
kappa = 10
optV = dict (intorder=10)
opt = dict (intorder=10)

In [ ]:
with TaskManager():
    V = HelmholtzSingleLayerPotentialOperator(fes_sphere, fes_sphere_test, kappa=kappa, **optV)
    C = HelmholtzCombinedFieldOperator(fes_sphere, fes_sphere_test, kappa=kappa, **opt)
    u = fes_sphere.TrialFunction()
    v = fes_sphere_test.TestFunction()
    Id = BilinearForm(u*v*ds).Assemble()

In [ ]:
lhs = 0.5 * Id.mat + C.mat
source = 1j * kappa * exp(1j * kappa * x) * (specialcf.normal(3)[0]-1)
rhs = LinearForm(source*v*ds).Assemble()

In [ ]:
gfu = GridFunction(fes_sphere)
pre = BilinearForm(u*v*ds, diagonal=True).Assemble().mat.Inverse(inverse="sparsecholesky")
with TaskManager():
    gfu.vec[:] = GMRes(A=lhs, b=rhs.vec, pre=pre, maxsteps=10)

In [ ]:
Draw (gfu, order=5, min=-5, max=5);

# prostprocessing on screen

In [ ]:
gf_screen = GridFunction(fes_screen)
print ("ndofscreen=", fes_screen.ndof)
with TaskManager():
    gf_screen.Set (V.GetPotential(gfu), definedon=mesh.Boundaries("screen"))

In [ ]:
scatter = mesh.BoundaryCF( {"screen":gf_screen- exp(1j * kappa * x)}, default=0)
print ("Total field")
Draw (scatter, mesh, min=-1,max=1, animate_complex=True, order=4);

In [ ]:
print ("Scattered field only:")
Draw (gf_screen, mesh, min=-1,max=1, animate_complex=True, order=4);